# NLP: Автоматическое построение рефератов текстовых документов.

In [2]:
!pip install -U transformers datasets sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [3]:
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [4]:
MAX_CHARS = 300
MODEL = "sarahai/ru-sum"

def clean_text(text):
    text = re.sub(r'[^а-яА-ЯёЁ0-9.,!?;:\-()\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

### Загрузка модели и токенизатора

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

### Применение модели -- генерация реферата

In [6]:
def generate_summary(text, max_chars=MAX_CHARS):
    text = clean_text(text)
    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
      summary_ids = model.generate(
          **inputs,
          max_length=150,
          min_length=30,
          num_beams=5,
          repetition_penalty=2.0,
          length_penalty=1.2,
          no_repeat_ngram_size=3,
          early_stopping=True
      )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary[:max_chars]

### Тестирование

In [7]:
texts = [
    "Вчера в Москве прошла выставка современных технологий. Посетители смогли увидеть роботов, дронов и новейшие разработки в сфере искусственного интеллекта.",
    "Учёные из Японии разработали метод восстановления изображений, которые видит человек, используя сигналы мозга. Эксперименты показали высокую точность восстановления."
]

for i, text in enumerate(texts, 1):
    summary = generate_summary(text)
    print(f"\n{i}:\n{summary}")


1:
В Москве прошла выставка современных технологий. Посетители смогли увидеть роботов и дронов, дронов и новейшие разработки в сфере искусственного интеллекта.

2:
Учёные из Японии разработали метод восстановления изображений, которые видит человек, используя сигналы мозга. Эксперименты показали высокую точность восстановления фотографий, которая видит человека. По ученым, учёные создали метод восстановление изображений на мозга, пользуя сигналами мозга в резу


### Оценка по метрикам ROUGE

In [39]:
texts = [
    "В Москве прошла выставка современных технологий. Посетители смогли увидеть роботов, дронов и новейшие разработки в сфере искусственного интеллекта.",
    "Учёные из Японии разработали метод восстановления изображений, которые видит человек, используя сигналы мозга. Эксперименты показали высокую точность восстановления.",
]

reference_summaries = [
    "В Москве прошла выставка современных технологий с роботами и дронами.",
    "Учёные из Японии восстановили изображения по сигналам мозга человека."
]

In [11]:
! pip install rouge

In [60]:
from rouge import Rouge

def evaluate_summary(summary, reference):
  for i, text in enumerate(reference):
      rouge = Rouge()
      scores = rouge.get_scores(summary[i], reference[i])[0]
      print(f"\nТекст {i}:\n{reference[i]}\n{summary[i]}")
      for rouge_type in ['rouge-1', 'rouge-2', 'rouge-l']:
        f_score = scores[rouge_type]['f']
        print(f"{rouge_type.upper()}: {f_score:.4f}")

In [61]:
summary = [generate_summary(text) for text in texts]
evaluate_summary(summary, reference_summaries)


Текст 0:
В Москве прошла выставка современных технологий с роботами и дронами.
В Москве прошла выставка современных технологий. Посетители смогли увидеть роботов и дронов, дронов и новейшие разработки в сфере искусственного интеллекта.
ROUGE-1: 0.4828
ROUGE-2: 0.3571
ROUGE-L: 0.4828

Текст 1:
Учёные из Японии восстановили изображения по сигналам мозга человека.
Учёные из Японии разработали метод восстановления изображений, которые видит человек, используя сигналы мозга. Эксперименты показали высокую точность восстановления фотографий, которая видит человека. По ученым, учёные создали метод восстановление изображений на мозга, пользуя сигналами мозга в резу
ROUGE-1: 0.2439
ROUGE-2: 0.0930
ROUGE-L: 0.2439


## Обработка примеров JSON

In [50]:
import json

In [62]:
with open("example_texts.json", "r", encoding="utf-8") as f:
    example_texts = json.load(f)

print(f"{len(example_texts)}")
print(example_texts[0][:400])

5
Увидеть мысль

Японским ученым удалось распознать изображение , увиденное человеком , сканируя его мозг .

Как пишет Yomiuri , группе сотрудников отделения нейроинформатики Международного исследовательского института передовых средств коммуникации ( Киото , Япония ) удалось восстановить изображение , увиденное человеком , опираясь только на сканирование электрических сигналов мозга .

В ходе экспе


In [63]:
generated_summaries = []

for i, text in enumerate(example_texts):
    summary = generate_summary(text)
    generated_summaries.append(summary)
    print(f"\n{i+1}:\n{summary}")


1:
«Увидеть мысль» Японским ученым удалось восстановить изображение, увиденное человеком, сканируя его мозг. В результате эксперимента исследователи продемонстрировали серию из 440 различных картинок, представлявших собой произвольное сочетание темных и светлых пятен на 100-пиксельном экране.

2:
Гидролизующий ацетилхолинэстеразное средство необратимого действия, акарицид широкого спектра действия. Применялся против комаров, клещей и клещей, паразитов, повреждающих фруктовые деревья, овощные деревья и декоративные растения.

3:
Власти Украины обещают сохранить двуязычие на востоке страны. На переговорах с главой МВД Арсением Яценюком и Ринатом Ахметовым в Донецке он предложил восстановить полномочия местных органов власти, которые будут избираться Областными советами.

4:
Московский мэр Москвы Борис Немцов заявил о том, что в прошлом году чиновники должны быть немедленно отправлен в отставку. По словам Михаила Лужкова, это может стать причиной того, что москвичи могут пойти на пенсию.

In [64]:
with open('generated_summaries.json', 'w', encoding='utf-8') as f:
      json.dump(generated_summaries, f, ensure_ascii=False, indent=4)

In [ ]:
with open("example_texts_summary.json", "r", encoding="utf-8") as f:
    example_texts_summary = json.load(f)

print(f"{len(example_texts_summary)}")
print(example_texts_summary[0])

5
Японские ученые смогли распознать изображение увиденное человеком сканируя его мозг. Восстановили изображение по сигналам мозговой активности. Показали картинки и измерили активность зрительной коры мозга.


In [69]:
evaluate_summary(generated_summaries, example_texts_summary)


Текст 0:
Японские ученые смогли распознать изображение увиденное человеком сканируя его мозг. Восстановили изображение по сигналам мозговой активности. Показали картинки и измерили активность зрительной коры мозга.
«Увидеть мысль» Японским ученым удалось восстановить изображение, увиденное человеком, сканируя его мозг. В результате эксперимента исследователи продемонстрировали серию из 440 различных картинок, представлявших собой произвольное сочетание темных и светлых пятен на 100-пиксельном экране.
ROUGE-1: 0.1786
ROUGE-2: 0.0727
ROUGE-L: 0.1786

Текст 1:
Карбофос - фосфорорганический инсектицид широкого спектра. Вытеснен пиретроидами, но эффективен против клопов. Действует через инактивацию ацетилхолинэстеразы.
Гидролизующий ацетилхолинэстеразное средство необратимого действия, акарицид широкого спектра действия. Применялся против комаров, клещей и клещей, паразитов, повреждающих фруктовые деревья, овощные деревья и декоративные растения.
ROUGE-1: 0.1538
ROUGE-2: 0.0526
ROUGE-L: 0.